# Imports and Constants

In [ ]:
lazy_copy = 'sample_results'

import pandas as pd
from tqdm import tqdm
import numpy as np
from datetime import datetime
import os
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from math import floor
import regex as re
from collections import Counter
from utils import prompt, update_labels, master_path, get_answer_vector, hash_dataframe
import json




mapping = lambda s: {'background': 'context',
                     'future_work': 'context',
                    'differences': 'context',
                     'future work': 'context',
                     'motivation': 'context',
                     'similarities': 'context',
                     'extends': 'extends',
                     '*': 'context'
                     }.get(s) or s

seed = 42 #np.random.randint(0, 100)

# Load existing results

In [ ]:

other_path = f'~/Desktop/2.FutureTech/uniform_sample/results/{lazy_copy.replace('/',':')}.csv'

df = pd.read_csv(other_path)
df = df.sort_values(by = 'multisentence').reset_index(drop = True)

df = df.sample(frac=1, random_state = seed)

df = df[df.index < 400]

print(len(df))
df.head()

In [ ]:
Counter(df['alex'])

# Parse outputs, generate confusion matrices

In [ ]:
background_mask = np.array([1,1,1,1,1,   0,0,0,0,0,   0,0,0,0,0,  1,1])
uses_mask = np.array([0,0,0,0,0,   0,1,1,1,1,   1,1,0,0,0,   0,0])
extends_mask = np.array([0,0,0,0,0,   0,0,0,0,0,   0,0,1,1,0,   0,0])


def get_truth(r, verbose = False, background_mask = background_mask, uses_mask = uses_mask, extends_mask = extends_mask):
    assert(len(background_mask) == len(uses_mask) and len(uses_mask) == len(extends_mask))
    answer_vector = get_answer_vector(r, length = len(background_mask))
    
    if answer_vector is None:
        return None
    
    resolve = lambda mask: np.array([{(0,1):0, (0,0): 0, (0, -1): 1, (1,1):1, (1,0) : 0, (1, -1): 0}[(x,y)] for x,y in zip(answer_vector, mask)]).sum()
    
    background =  resolve(background_mask)
    uses = resolve(uses_mask)
    fine_tune = resolve(extends_mask)

    
    if (verbose):
        print(answer_vector)
        print(f"bg: {background}, uses: {uses}, extends: {fine_tune}", flush = True)
        
    if answer_vector[3]:
        return 'background'

    if fine_tune:
        return 'extends'
    
    if uses:
        return 'uses'
    
    return 'background'

#df['gpt-vector'] = df[label_col].apply(get_truth)
#print(set(df['gpt-vector']))

for idx, row in df.iterrows():
    continue
    if (row['gpt-vector'] is None):
        print(row['multisentence'])
        print(row['gpt-vector'])
        print("\n")

In [ ]:
y_label, x_label = 'alex', 'learned_classification'

In [ ]:
tt_split = 2/3
split = floor(tt_split * len(df))
test_mode = False  

true = df[y_label][split:] if test_mode else df[y_label][:split]
pred = df[x_label][split:] if test_mode else df[x_label][:split]

print(f"Labels hash: {hash_dataframe(list(true))}")
print(f"Class hash: {hash_dataframe(list(pred))}")


true, pred = list(map(mapping, true)), list(map(mapping, pred))



cm = confusion_matrix(true, pred)
print(set(true), set(pred))
labels = sorted(list({mapping(item) for item in ['background', 'extends', 'uses']}))

diagonal = cm * np.identity(len(labels))
false_positives = (cm - diagonal).sum(axis = 0)/cm.sum(axis = 0)
false_negatives = (cm - diagonal).sum(axis = 1)/cm.sum(axis = 1)
accuracy = diagonal.sum() / cm.sum()

print(f"Accuracy {accuracy}\n")
print(f"Uses false positive: {false_positives[-1]}")
print(f"Uses false negative: {false_negatives[-1]}\n")
print(f"Extends false positive: {false_positives[1]}")
print(f"Extends false negative: {false_negatives[1]}")


sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title("Test Set Confusion Matrix with Fixed Threshold")
plt.xlabel(x_label)
plt.ylabel(y_label)


In [ ]:
raise Exception

# Inspect Confusion Matrix
## Get samples, option to relabel

In [ ]:
def get_examples(y_value, x_value):
    mask = (df[y_label].apply(mapping) == y_value) & (df[x_label].apply(mapping) == x_value) 
    return df[mask]

idx = -1
samples = get_examples(y_value='context', x_value = 'uses')
samples.head()

In [ ]:
index = samples.index[idx]
print(idx, index)
print(f"Drawing from {len(samples)} samples")
print(samples['modelKey'].iloc[idx])

print(samples['multisentence'].iloc[idx])

vector = [float(el) for el in samples['answer_vector'].iloc[idx][1:-1].split(' ')]
print(vector)
one_indices = [i for i in range(len(vector)) if vector[i] == 1]
one_idx = 1
print(one_indices[one_idx], json.loads(samples['answer_string'].iloc[idx])[one_indices[one_idx]])

print(samples[y_label].iloc[idx])
print(samples[x_label].iloc[idx])

In [ ]:
df_temp = pd.read_csv(master_path + '.csv')
assert(df_temp['multisentence'].loc[index] == df['multisentence'].loc[index])
print(df_temp['multisentence'].loc[index])
reclass = input()

if (len(reclass) > 0):
    df_temp['alex2'].loc[index] = reclass
    df_temp.to_csv(master_path + '.csv', index = False)

# Reprompt on samples

In [ ]:
retry_sample = True
if (retry_sample):
    for idx in tqdm(samples.index):    
        df[label_col].at[idx] = prompt(
                                    MULTIPROMPT.format(
                                            input = df['multisentence'].loc[idx], 
                                            modelKey = df['modelKeyStriped'].loc[idx]))
CONNECTION_ON = False

In [ ]:
Counter([df['gpt-vector'].loc[i] for i in samples.index])